In [78]:
import numpy as np
from Pyfhel import Pyfhel, PyCtxt

In [79]:
def nomralize(x):
    return (x - np.mean(x)) / np.std(x)

def encrypt_data(data_provider, data):
    s_encrypted_data = np.ndarray(data.shape, dtype=PyCtxt)
    for i in range(len(data)):
        for j in range(len(data[i])):
            s_encrypted_data[i][j] = data_provider.encrypt(data[i][j])
    return s_encrypted_data

def decrypt_data(data_provider, data):
    s_decrypted_data = np.ndarray(data.shape, dtype=float)
    for i in range(len(data)):
        for j in range(len(data[i])):
            s_decrypted_data[i][j] = data_provider.decrypt(data[i][j])[0]
    return s_decrypted_data

In [80]:
def HEmatmult(a, b):
    return np.matmul(a, b)

def HEreduce_mean(a, axis=None):
    if axis is None:
        return np.sum(a)/a.size
    return np.sum(a, axis=axis)/a.shape[axis]

def HEreduce_sum(a, axis=None):
    return np.sum(a, axis=axis)

def HEexp(a, n=4):
    d = 1.0
    rt = 0.0
    for i in range(n):
        rt += d/np.math.factorial(i)
        d = a*d
    return rt

def HEsquare(a):
    return a**2

In [81]:
c1 = Pyfhel(context_params={'scheme':'ckks', 'n':2**13, 'scale':2**30, 'qi_sizes':[30]*5})
# generate key pair
c1.keyGen()
c1

<ckks Pyfhel obj at 0x21eed60d840, [pk:Y, sk:Y, rtk:-, rlk:-, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>

In [82]:
server = Pyfhel()
server.from_bytes_context(c1.to_bytes_context())
server.from_bytes_public_key(c1.to_bytes_public_key())
server

<ckks Pyfhel obj at 0x21ee61bf000, [pk:Y, sk:-, rtk:-, rlk:-, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>

In [83]:
# Use c1 to encrypt
test1 = np.array([[1, 2, 3], [4, 5, 6]], dtype=float)
test1_encrypt = encrypt_data(c1, test1)
test1_encrypt

array([[<Pyfhel Ciphertext at 0x21ee62086d0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
        <Pyfhel Ciphertext at 0x21ee62c8ef0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
        <Pyfhel Ciphertext at 0x21eed628a90, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>],
       [<Pyfhel Ciphertext at 0x21eed6290d0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
        <Pyfhel Ciphertext at 0x21ee62c0d10, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
        <Pyfhel Ciphertext at 0x21eed60f9c0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>]],
      dtype=object)

In [84]:
# use c1 to decrypt
test1_decrypt = decrypt_data(c1, test1_encrypt)
test1_decrypt

array([[0.99999841, 2.00000098, 3.00000139],
       [3.99999994, 4.99999909, 5.99999958]])

## Caculation that model need

In [85]:
sum = HEreduce_sum(test1_encrypt)
mean = HEreduce_mean(test1_encrypt)
square = HEsquare(test1_encrypt)

In [86]:
server_array = np.array([[3, 2], [6, 5], [1, 4]], dtype=float)
encoded_array = server_array
for i in range(len(server_array)):
    for j in range(len(server_array[i])):
        encoded_array[i][j] = server.encode(server_array[i][j])

In [87]:
mult = HEmatmult(test1_encrypt, server_array)

## Verify Result

In [88]:
print("Sum:", c1.decrypt(sum)[0], "vs", np.sum(test1))
print("Mean:", c1.decrypt(mean)[0], "vs", np.mean(test1))
print("Square:", decrypt_data(c1, square), "vs", test1**2)
print("Matmul:", decrypt_data(c1, mult), "vs", np.matmul(test1, server_array))

Sum: 20.999999380423812 vs 21.0
Mean: 3.4999995389890155 vs 3.5
Square: [[ 0.99999682  4.00000391  9.00000836]
 [15.99999953 24.99999085 35.99999491]] vs [[ 1.  4.  9.]
 [16. 25. 36.]]
Matmul: [[18.00000248 24.00000727]
 [47.99999391 56.99999361]] vs [[18. 24.]
 [48. 57.]]
